In [47]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import DirectoryIterator, ImageDataGenerator

### Model training

In [ ]:
training_set = DirectoryIterator(directory = 'Large-image-dataset/training',
                                 image_data_generator = ImageDataGenerator(rescale=1.0/255),
                                 target_size = (64, 64),
                                 batch_size = 32,
                                 shuffle = False,  
                                 class_mode = 'binary')

Found 2246 images belonging to 2 classes.


In [49]:
print(training_set.classes)
unique, counts = np.unique(training_set.classes, return_counts=True)
print(dict(zip(training_set.class_indices, counts)))

[0 0 0 ... 1 1 1]
{'Legal': 773, 'No ball': 1473}


In [50]:
print(training_set.samples)

2246


In [51]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
import numpy as np

# Initialize empty lists for training data
X_train = []
y_train = []

# Iterate through the training set and collect the data
i=1
for x, y in training_set:
    print(i)
    i+=1
    X_train.extend(x)
    y_train.extend(y)
    if len(X_train) >= training_set.samples:
        break
# Convert the lists into numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape the image data to a 2D array (flatten each image)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE to the reshaped training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_reshaped, y_train)

# Reshape the resampled data back to its original 4D shape (64x64 images with 3 color channels)
X_resampled = X_resampled.reshape(-1, 64, 64, 3)

# Shuffle the resampled data
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

# Now you have X_resampled and y_resampled with balanced classes

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71


In [52]:
new_datagen = ImageDataGenerator(rescale=1./255)

# Create the new training set using the resampled data
training_set_resampled = new_datagen.flow(X_resampled, y_resampled, batch_size=32)
# Convert X_resampled and y_resampled into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))

# Shuffle, batch, and optionally prefetch the data for efficient training
training_set_smote = train_dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Use this dataset in your model training

In [53]:
print(y_resampled)
unique, counts = np.unique(y_resampled, return_counts=True)
print(dict(zip(unique, counts)))

[0. 1. 1. ... 1. 1. 1.]
{0.0: 1473, 1.0: 1473}


In [ ]:
validation_set = DirectoryIterator(directory = 'Large-image-dataset/validation',
                                   image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                   target_size = (64, 64),
                                   batch_size = 32,
                                   shuffle = False,
                                   class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [55]:
cnn = tf.keras.models.Sequential()

In [56]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=(64, 64, 3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [57]:
cnn.add(tf.keras.layers.Flatten())

In [58]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [59]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [60]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [61]:
cnn.fit(x = training_set_smote, validation_data = validation_set, epochs = 10)

Epoch 1/10
93/93 [==============================] - 12s 109ms/step - loss: 0.6545 - accuracy: 0.7261 - val_loss: 0.4767 - val_accuracy: 0.7824
Epoch 2/10
93/93 [==============================] - 9s 101ms/step - loss: 0.2820 - accuracy: 0.8924 - val_loss: 0.8685 - val_accuracy: 0.6031
Epoch 3/10
93/93 [==============================] - 10s 112ms/step - loss: 0.2715 - accuracy: 0.9005 - val_loss: 0.2903 - val_accuracy: 0.9046
Epoch 4/10
93/93 [==============================] - 10s 109ms/step - loss: 0.1148 - accuracy: 0.9657 - val_loss: 0.1879 - val_accuracy: 0.9618
Epoch 5/10
93/93 [==============================] - 10s 107ms/step - loss: 0.0607 - accuracy: 0.9861 - val_loss: 0.1740 - val_accuracy: 0.9771
Epoch 6/10
93/93 [==============================] - 11s 118ms/step - loss: 0.0338 - accuracy: 0.9966 - val_loss: 0.1747 - val_accuracy: 0.9733
Epoch 7/10
93/93 [==============================] - 10s 107ms/step - loss: 0.0211 - accuracy: 0.9983 - val_loss: 0.1820 - val_accuracy: 0.9695


In [62]:
cnn.save('final.keras')

### Model testing

In [ ]:
from sklearn.metrics import classification_report

loaded_model = tf.keras.models.load_model(r'trial.keras')

In [ ]:
testing_set = DirectoryIterator(directory = 'Large-image-dataset/testing',
                                image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                target_size = (64, 64),
                                batch_size = 32,
                                shuffle = False,
                                class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [65]:
loss, accuracy = loaded_model.evaluate(training_set)

print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

y_pred = loaded_model.predict(training_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(training_set.classes.tolist(), y_pred))

71/71 [==============================] - 26s 364ms/step - loss: 0.0088 - accuracy: 1.0000
Training Loss: 0.00876210443675518
Training Accuracy: 1.0
71/71 [==============================] - 24s 337ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       773
           1       1.00      1.00      1.00      1473

    accuracy                           1.00      2246
   macro avg       1.00      1.00      1.00      2246
weighted avg       1.00      1.00      1.00      2246



In [66]:
loss, accuracy = loaded_model.evaluate(validation_set)

print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

y_pred = loaded_model.predict(validation_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(validation_set.classes.tolist(), y_pred))

9/9 [==============================] - 3s 351ms/step - loss: 0.1791 - accuracy: 0.9771
Validation Loss: 0.1791359782218933
Validation Accuracy: 0.9770992398262024
9/9 [==============================] - 3s 354ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       129
           1       0.96      0.99      0.98       133

    accuracy                           0.98       262
   macro avg       0.98      0.98      0.98       262
weighted avg       0.98      0.98      0.98       262



In [67]:
loss, accuracy = loaded_model.evaluate(testing_set)

print("Testing Loss:", loss)
print("Testing Accuracy:", accuracy)

y_pred = loaded_model.predict(testing_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(testing_set.classes.tolist(), y_pred))

9/9 [==============================] - 3s 374ms/step - loss: 0.1791 - accuracy: 0.9771
Testing Loss: 0.1791359782218933
Testing Accuracy: 0.9770992398262024
9/9 [==============================] - 3s 377ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       129
           1       0.96      0.99      0.98       133

    accuracy                           0.98       262
   macro avg       0.98      0.98      0.98       262
weighted avg       0.98      0.98      0.98       262



In [68]:
print(testing_set.classes.tolist())
print(y_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [69]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [70]:
# Load the test image
image_location = r'test\legal.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 178ms/step
[[0.]]
Legal Ball


In [71]:
# Load the test image
image_location = r'test\noball_1.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 60ms/step
[[0.9961649]]
No Ball


In [72]:
# Load the test image
image_location = r'test\noball_2.jpg'

test_image = image.load_img(image_location, target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

# Now you can use the loaded_model for predictions or further training
result = loaded_model.predict(test_image)
print(result)

if result[0][0] < 0.5:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 43ms/step
[[1.]]
No Ball


In [ ]:
testing_set = DirectoryIterator(directory = 'Small-image-dataset',
                                image_data_generator = ImageDataGenerator(rescale = 1.0/255),
                                target_size = (64, 64),
                                batch_size = 32,
                                shuffle = False,
                                class_mode = 'binary')

Found 77 images belonging to 2 classes.


In [74]:
loss, accuracy = loaded_model.evaluate(testing_set)

print("Testing Loss:", loss)
print("Testing Accuracy:", accuracy)

y_pred = loaded_model.predict(testing_set)
y_pred = [1 if i[0] >= 0.5 else 0 for i in y_pred]

print(classification_report(testing_set.classes.tolist(), y_pred))

3/3 [==============================] - 1s 68ms/step - loss: 1.6655 - accuracy: 0.7013
Testing Loss: 1.6655004024505615
Testing Accuracy: 0.701298713684082
3/3 [==============================] - 1s 152ms/step
              precision    recall  f1-score   support

           0       0.54      0.52      0.53        25
           1       0.77      0.79      0.78        52

    accuracy                           0.70        77
   macro avg       0.66      0.65      0.66        77
weighted avg       0.70      0.70      0.70        77



In [77]:
print(tf.version.VERSION)

2.10.0
